In [3]:
# required libraries
import pandas as pd
import numpy as np
import librosa
from joblib import dump, load
import keras

In [4]:
# loading normalizer
scaler = load('/content/std_scaler.bin')

# loading preprocessor function
def preprocess(fn_wav):
    y, sr = librosa.load(fn_wav, mono=True, duration=5)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    
    feature_row = {        
        'chroma_stft': np.mean(chroma_stft),
        'rmse': np.mean(rmse),
        'spectral_centroid': np.mean(spectral_centroid),
        'spectral_bandwidth': np.mean(spectral_bandwidth),
        'rolloff': np.mean(rolloff),
        'zero_crossing_rate': np.mean(zcr),        
    }
    for i, c in enumerate(mfcc):
        feature_row[f'mfcc{i+1}'] = np.mean(c)

    return feature_row

In [5]:
#Features to be extracted
features =  [
                'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate',
                'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 
                'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'
            ]

class_names = ['Covid', 'Non-Covid']

In [6]:
fn_wav_pos = '/content/pos-0421-084-cough-m-50-4.mp3' # positive example
df = pd.DataFrame(columns = features)
df = df.append(preprocess(fn_wav_pos), ignore_index = True)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [7]:
X_test = scaler.transform(df)
X_test

array([[-0.79976089,  0.09133042, -0.93914436, -0.74289653, -0.82792065,
        -1.00167803,  0.08233783, -0.14509999,  0.13946402,  0.29447243,
         0.17638311, -0.3114036 , -0.59769858, -0.36673527, -1.21149372,
         0.47812642,  1.80836824, -0.74647968,  0.46772948, -0.53344665,
         1.18344649,  0.55849719, -0.75789379,  0.19436788, -0.51917186,
        -1.13911157]])

In [9]:
model = keras.models.load_model('/content/save_at_16.h5')
pred = model.predict(X_test)[0]

#final
predicted = class_names[int(np.round(pred)[0])]
confidence = pred[0]
print('Result: {}\nConfidence: {}'.format(predicted, max(1 - confidence, confidence)))

Result: Covid
Confidence: 0.9998960817902116


# Converting the model to tflite format

In [10]:
import tensorflow as tf
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file('/content/save_at_16.h5') # path to the SavedModel directory
tflite_model = converter.convert()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


INFO:tensorflow:Assets written to: /tmp/tmpps92b72w/assets
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /tmp/tmpps92b72w/variables/variables
INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}
INFO:tensorflow:input tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: input_16
INFO:tensorflow: tensor name: serving_default_input_16:0, shape: (-1, 26), type: DT_FLOAT
INFO:tensorflow:output tensors info: 
INFO:tensorflow:Tensor's key in saved_model's tensor_map: dense_99
INFO:tensorflow: tensor name: StatefulPartitionedCall:0, shape: (-1, 1), type: DT_FLOAT
INFO:tensorflow:Restoring parameters from /tmp/tmpps92b72w/variables/variables
Inst

In [11]:
# saving the converted model
open('model_tflite_tabular_audio.tflite', "wb").write(tflite_model)

750792

# TFLite Inference

In [12]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [13]:
fn_wav_pos = '/content/pos-0421-084-cough-m-50-4.mp3' # positive example
df = pd.DataFrame(columns = features)
df = df.append(preprocess(fn_wav_pos), ignore_index = True)
X_test = scaler.transform(df)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [14]:
input_shape = input_details[0]['shape']
input_data = np.array(X_test, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

#final
predicted = class_names[int(np.round(output_data[0][0]))]
confidence = output_data[0][0]
print('Result: {}\nConfidence: {}'.format(predicted, max(1 - confidence, confidence)))

Result: Covid
Confidence: 0.9998960817902116
